<h1>Google Pennsylvania Review Ingestion</h1>

In [2]:
import pandas as pd 
import numpy as np 
import json 
import uuid
from datetime import datetime

## Ingest Review Data and turn into CSV

In [6]:
raw_data_path = './data/review-Pennsylvania_10.json'

In [7]:
def ingest_reviews_data(reviews_path): 
    data_list = [] 
    with open(reviews_path, 'r') as f: 
        print("Starting Ingestion")
        for line in f: 
            data = json.loads(line)
            if not data: 
                continue
            review_id = uuid.uuid4()
            gmap_id = data['gmap_id']
            rating = data['rating']
            pics = data['pics']
            user_id = data['user_id']
            name = data['name'] 
            time_unix =  data['time'] 
            rating = data['rating']
            text = data['text'] 
            
            #convert timestamp to seconds datetime
            time = datetime.fromtimestamp(time_unix/1000) if time_unix else None
            data_list.append([review_id, gmap_id, rating, pics, user_id, name, time, text])
        print("Finished Ingestion")
    df = pd.DataFrame(data_list, columns=['review_id', 'gmap_id', 'rating', 'pics', 'user_id', 'name', 'time', 'text'])
    return df 


In [8]:
df_reviews = ingest_reviews_data(raw_data_path)

Starting Ingestion
Finished Ingestion


In [11]:
df_reviews.head()

,review_id,gmap_id,rating,pics,user_id,name,time,text
0,b05994c1-edfd-4375-b9c8-1aa60f929eb0,0x89c6c63c8cd87141:0x54d0d283872eecbb,5,None,104944742559079759337,Jaron Whitfield,2018-02-04 03:09:22.839,"Joe is quite unique of his line of work, he as..."
1,e0cbc10b-18d9-48ba-8383-f1e99b34c51c,0x89c6c63c8cd87141:0x54d0d283872eecbb,5,None,117609702832980320437,Jonathan McCarthy,2016-10-12 08:44:51.163,For such a small place their impact on my life...
2,35a2f2a7-148a-49e2-930c-7cf9367d8c91,0x89c6c63c8cd87141:0x54d0d283872eecbb,5,None,110563242018426632956,Rocky Kev,2012-06-04 12:22:25.578,I usually give them a call before I stop by to...
3,af853df6-9aa0-430d-acd6-59f553faa47e,0x89c6c63c8cd87141:0x54d0d283872eecbb,5,None,112895973505401399406,Josep Valls,2013-03-14 14:35:10.554,My bike had been sitting outdoors for a good w...
4,0cf4a38f-88ad-4c69-b0c9-e42016962934,0x89c6c63c8cd87141:0x54d0d283872eecbb,5,None,110619674885963828884,Timaree Schmit,2019-01-29 16:45:29.760,Always an easy experience. Service is knowledg...


In [12]:
df_reviews.describe()

,rating,time
count,1.277236e+07,12772358
mean,4.293200e+00,2019-02-19 10:12:45.293065984
min,1.000000e+00,1990-12-30 19:00:00
25%,4.000000e+00,2018-05-03 10:10:00.285250048
50%,5.000000e+00,2019-03-17 22:27:47.787500032
75%,5.000000e+00,2019-12-23 20:01:15.136249856
max,5.000000e+00,2021-09-08 16:37:19.739000
std,1.062594e+00,NaN


In [15]:
df_reviews.to_csv('./data/ingested/reviews.csv', index=False, escapechar='\\')

## Process Pics from reviews

In [17]:
reviews_df = pd.read_csv('./data/ingested/reviews.csv', escapechar='\\')

In [18]:
reviews_df.head()

,review_id,gmap_id,rating,pics,user_id,name,time,text
0,b05994c1-edfd-4375-b9c8-1aa60f929eb0,0x89c6c63c8cd87141:0x54d0d283872eecbb,5,NaN,104944742559079759337,Jaron Whitfield,2018-02-04 03:09:22.839,"Joe is quite unique of his line of work, he as..."
1,e0cbc10b-18d9-48ba-8383-f1e99b34c51c,0x89c6c63c8cd87141:0x54d0d283872eecbb,5,NaN,117609702832980320437,Jonathan McCarthy,2016-10-12 08:44:51.163,For such a small place their impact on my life...
2,35a2f2a7-148a-49e2-930c-7cf9367d8c91,0x89c6c63c8cd87141:0x54d0d283872eecbb,5,NaN,110563242018426632956,Rocky Kev,2012-06-04 12:22:25.578,I usually give them a call before I stop by to...
3,af853df6-9aa0-430d-acd6-59f553faa47e,0x89c6c63c8cd87141:0x54d0d283872eecbb,5,NaN,112895973505401399406,Josep Valls,2013-03-14 14:35:10.554,My bike had been sitting outdoors for a good w...
4,0cf4a38f-88ad-4c69-b0c9-e42016962934,0x89c6c63c8cd87141:0x54d0d283872eecbb,5,NaN,110619674885963828884,Timaree Schmit,2019-01-29 16:45:29.760,Always an easy experience. Service is knowledg...


In [19]:
reviews_df.describe()

,rating
count,1.277236e+07
mean,4.293200e+00
std,1.062594e+00
min,1.000000e+00
25%,4.000000e+00
50%,5.000000e+00
75%,5.000000e+00
max,5.000000e+00


Make sure that we don't include any records that have both text and pics null <br>

Filter based on desired criteria

In [20]:
filtered_reviews_df = reviews_df[(reviews_df['pics'].notna()) | (reviews_df['text'].notna())]

In [21]:
filtered_null = reviews_df[(reviews_df['pics'].isna()) & (reviews_df['text'].isna())]

In [22]:
print(len(filtered_null))

5936425


In [23]:
print(len(filtered_reviews_df))

6835933


In theory should add up to the length of the total dataset so we know we've done our process correct:

In [24]:
total_reviews = len(reviews_df)
total_added = len(filtered_reviews_df) + len(filtered_null)

print("The total reviews: ")
print(total_reviews)

print("The total reviews between the null and not null: ")
print(total_added)

print("Difference: ")
print(total_reviews - total_added)

The total reviews: 
12772358
The total reviews between the null and not null: 
12772358
Difference: 
0


In [25]:
filtered_reviews_df.describe()

,rating
count,6.835933e+06
mean,4.277457e+00
std,1.140362e+00
min,1.000000e+00
25%,4.000000e+00
50%,5.000000e+00
75%,5.000000e+00
max,5.000000e+00


In [28]:
filtered_reviews_df.to_csv('./data/ingested/reviews_filtered.csv', index=False, escapechar='\\')

In [27]:
import ast
def extract_pic_urls(row): 
    if pd.notna(row['pics']): 
        try:
            pics_list = ast.literal_eval(row['pics'])
            return [{'review_id':row['review_id'], 'pic_url': url} for pic_dict in pics_list for url in pic_dict.get('url',[])]
        except Exception as e: 
            print(e)
            return [{'review_id': row['review_id'], 'pic_url': None}]
    else: 
        return [{'review_id': row['review_id'], 'pic_url': None}]

In [29]:
pics_df = pd.DataFrame([item for sublist in filtered_reviews_df.apply(extract_pic_urls, axis=1) for item in sublist])

In [30]:
pics_df.to_csv('./data/ingested/pics.csv', index=False, escapechar='\\')